In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import random

# Reading data & buliding the graph

In [4]:
# nodes_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graph_data/jalili_nodes.csv')
# edges_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/graph_data/jalili_edges.csv')

nodes_df = pd.read_csv('../OneDrive/Documents/jalili_nodes.csv')
edges_df = pd.read_csv('../OneDrive/Documents/jalili_edges.csv')

G = nx.DiGraph()

# اضافه کردن نودها با ویژگی‌ها
for _, row in nodes_df.iterrows():
    G.add_node(row['Id'],
               Label=row['Label'],
               views=row['views'],
               followers=row['followers'],
               importance=row['importance'],
               posts=row['posts'],
               modularity_class=row['modularity_class'],
               indegree=row['indegree'],
               outdegree=row['outdegree'],
               Degree=row['Degree'],
               closeness_centrality=row['closnesscentrality'],
               betweenness_centrality=row['betweenesscentrality'])

# اضافه کردن یال‌ها با وزن
for _, row in edges_df.iterrows():
    G.add_edge(row['Source'], row['Target'], weight=row['Weight'], candidate=row['Candidate'])


In [5]:
nodes_df

,Id,Label,views,followers,importance,posts,indegree,outdegree,Degree,closnesscentrality,betweenesscentrality,modularity_class
0,1025087215,volleyballpress,11237,4389,1.417582,17,0,1,1,0.184932,0.000000,45
1,1447659114,gazersesi,3049,1160,1.268100,37,0,2,2,0.666667,0.000000,37
2,1543593190,makos_herat,2979,6272,1.145952,7,0,1,1,0.176601,0.000000,168
3,1826268931,sedalihasani,89823,6051,2.333737,47,3,0,3,0.000000,0.000000,0
4,1124128736,alisaraye_fouman,0,114,1.025306,6,0,1,1,1.000000,0.000000,139
...,...,...,...,...,...,...,...,...,...,...,...,...
8621,1055964609,jokandannews,19189,5106,1.683858,41,4,3,7,0.193103,13035.964877,37
8622,1002276548,ale_yaasin,17612,3041,2.009541,148,0,5,5,0.218336,0.000000,139
8623,1264367454,darhamnewsiran,201,61,1.327344,107,0,14,14,0.247943,0.000000,189
8624,1407302874,rahmaatism,218422,14313,4.037705,415,26,11,37,0.255401,25321.984347,139


In [ ]:
incoming_edges = G.in_edges(1407302874, data=True)
incoming_edges

In [6]:
def compute_spread_probabilities(G, node):
    incoming_edges = G.in_edges(node, data=True)
    total_weight = sum(edge_data['weight'] for _, _, edge_data in incoming_edges)
    probabilities = {}

    for source, _, edge_data in incoming_edges:
        probabilities[source] = edge_data['weight'] / total_weight

    return probabilities

In [71]:
def find_by_label(label, G):
    for node, attrs in G.nodes(data=True):
        if attrs.get('Label') == label:
            return node
    return None

def find_by_modularity_class(modularity_class, G):
    community = []
    for node, attrs in G.nodes(data=True):
        if attrs.get('modularity_class') == modularity_class:
            community.append(node)
    return community


def find_top_high_measure(G, number, measure):
    sorted_nodes = sorted(G.nodes(data=True), key=lambda x: x[1][measure], reverse=True)
    return [node for node,  _ in sorted_nodes[:number]]

def find_top_high_measure_list(node_list, G, number, measure):
    sorted_nodes = sorted(node_list, key=lambda node_id: G.nodes[node_id].get(measure, 0), reverse=True)
    return sorted_nodes[:number]


def find_refference_nodes(G, indegree_limit):
    refference_nodes = [node for node, attrs in G.nodes(data=True) if attrs['indegree'] > indegree_limit]
    return refference_nodes

def find_top_bridges(G, bridge_nodes):
    sorted_nodes = sorted(bridge_nodes, key=lambda node_id: G.nodes[node_id].get('indegree', 0), reverse=True)
    return sorted_nodes

In [70]:
find_top_high_measure_list( [1027676885,1005280116,  1005866359, 1006227643, 1532857131, 1069291200, 1002843824, 1454361599, 1007884158, 1002062924, 1285183655, 1127390421, 1426015854, 1741977413, 1489742853, 1181983958, 1258023043, 1990273594, 1093801324, 1106958042, 1357363429, 1364175336, 1270042187, 1108334723, 1072766900],G, 25,'betweenness_centrality')

1027676885 : 1207116.669277
1005866359 : 645405.185361
1005280116 : 609338.628452
1006227643 : 557287.079398
1532857131 : 517923.729811
1069291200 : 474058.231487
1002843824 : 350681.580023
1454361599 : 311612.213778
1007884158 : 309020.922166
1002062924 : 298345.213522
1285183655 : 294707.936566
1127390421 : 285415.676094
1426015854 : 278755.34286
1741977413 : 275680.031856
1489742853 : 255916.886756
1181983958 : 242346.322421
1258023043 : 238285.487976
1990273594 : 176115.971128
1093801324 : 170622.682795
1106958042 : 160429.286611
1357363429 : 158419.907303
1364175336 : 157283.743212
1270042187 : 154974.200679
1108334723 : 144995.681729
1072766900 : 144264.769374


[1027676885,
 1005866359,
 1005280116,
 1006227643,
 1532857131,
 1069291200,
 1002843824,
 1454361599,
 1007884158,
 1002062924,
 1285183655,
 1127390421,
 1426015854,
 1741977413,
 1489742853,
 1181983958,
 1258023043,
 1990273594,
 1093801324,
 1106958042,
 1357363429,
 1364175336,
 1270042187,
 1108334723,
 1072766900]

In [8]:
import networkx as nx
import matplotlib.pyplot as plt

def draw_multiple_graphs(inputs, iteration, layout_func=nx.spring_layout):
    num_graphs = len(inputs)
    plt.figure(figsize=(5 * num_graphs, 5)) 

    for i, input in enumerate(inputs):
        G, visited_nodes = input
        pos = nx.random_layout(G)

        plt.subplot(1, num_graphs, i + 1)

        active_nodes = [node for node in G.nodes() if node in visited_nodes]
        inactive_nodes = [node for node in G.nodes() if node not in visited_nodes]
        active_edges = [(u, v) for u, v in G.edges() if u in visited_nodes and v in visited_nodes]

        nx.draw_networkx_nodes(G, pos, nodelist=active_nodes, node_color='red', node_size=30, label='Active')
        nx.draw_networkx_nodes(G, pos, nodelist=inactive_nodes[:50], node_color='blue', node_size=30, label='Inactive')
        nx.draw_networkx_edges(G, pos, edgelist=active_edges, alpha=0.5, edge_color='gray')

        labels = {node: G.nodes[node]['Label'] for node in active_nodes if 'Label' in G.nodes[node]}
        print(labels)
        nx.draw_networkx_labels(G, pos, labels=labels, font_size=8)

        plt.title(f"Graph {i + 1}")

    plt.show()

In [143]:
import matplotlib.pyplot as plt
import copy
import math

    
def simulate_spread(G, start_node,  disabled_nodes = None, visualization=False , iterations=10):
    reach_counts = []

    G_copy = G.copy()
    if disabled_nodes:
        G_copy.remove_nodes_from(disabled_nodes)
        
    for iteration in range(iterations):
        if visualization:
            print(f"---------iteration {iteration}--------------")
        visited = set()
        queue = [start_node]
        graphs = []
        reach_count = 0

        while queue:
            current_node = queue.pop(0)
            if current_node not in visited and G_copy.has_node(current_node):
                visited.add(current_node)
                reach_count += 1
                
                spread_probs = compute_spread_probabilities(G_copy, current_node)

                for neighbor, prob in spread_probs.items():
                    if random.random() < prob:  
                        queue.append(neighbor)

            gr = G_copy.copy()
            vi = visited.copy()
            graphs.append([gr, vi])

        if visualization:
            draw_multiple_graphs(graphs, iteration)
        reach_counts.append(reach_count)
        
    return reach_counts

In [143]:
import matplotlib.pyplot as plt
import copy
import math

    
def simulate_spread(G, start_node,  disabled_nodes = None, visualization=False , iterations=10):
    reach_counts = []

    G_copy = G.copy()
    if disabled_nodes:
        G_copy.remove_nodes_from(disabled_nodes)
        
    for iteration in range(iterations):
        if visualization:
            print(f"---------iteration {iteration}--------------")
        visited = set()
        queue = [start_node]
        graphs = []
        reach_count = 0

        while queue:
            current_node = queue.pop(0)
            if current_node not in visited and G_copy.has_node(current_node):
                visited.add(current_node)
                reach_count += 1
                
                spread_probs = compute_spread_probabilities(G_copy, current_node)

                for neighbor, prob in spread_probs.items():
                    if random.random() < prob:  
                        queue.append(neighbor)

            gr = G_copy.copy()
            vi = visited.copy()
            graphs.append([gr, vi])

        if visualization:
            draw_multiple_graphs(graphs, iteration)
        reach_counts.append(reach_count)
        
    return reach_counts

In [125]:
from itertools import islice

def find_top_high_fashaness(G, start_node, number):
    top_high_fashaness = set()

    community_hbt_share = round(0.5 * number)
    community_brg_share = round(0.4 * number)
    totalgraph_hbt_share = round(0.1 * number)

    # Share from target community top betweenness nodes
    print("Target community: ", G.nodes[start_node]['modularity_class'])
    target_community_nodes = find_by_modularity_class(G.nodes[start_node]['modularity_class'], G)
    target_community_nodes.remove(start_node)
    community_hbt_share = min(community_hbt_share, len(target_community_nodes))
    tops = find_top_high_measure_list(target_community_nodes, G, community_hbt_share ,'betweenness_centrality')
    top_high_fashaness.update(tops)
    
    # Share from community bridge nodes
    all_possible_bridges = set()
    bridge_nodes = []
    for node in target_community_nodes:
        incoming_neighbors = list(G.predecessors(node))
        all_possible_bridges.update(incoming_neighbors)
    for node in all_possible_bridges:
        if node not in target_community_nodes:
            bridge_nodes.append(node)
    bridge_nodes = find_top_bridges(G, bridge_nodes)
    community_brg_share = min(community_brg_share, len(bridge_nodes))
    top_high_fashaness.update(
        node for node in bridge_nodes
        if node not in top_high_fashaness and len(top_high_fashaness) <= (community_brg_share + community_hbt_share)
    )
    
    # Share from total graph top betweenness nodes
    tops = find_top_high_measure(G, len(G.nodes()) ,'betweenness_centrality')
    top_high_fashaness.update(
        node for node in tops
        if node not in top_high_fashaness and len(top_high_fashaness) <= number
    )
    
    return list(top_high_fashaness)

In [128]:
from itertools import islice

def find_top_high_fashaness1(G, start_node, number):
    top_high_fashaness = set()

    community_hbt_share = round(0.5 * number)
    community_brg_share = round(0.4 * number)
    totalgraph_hbt_share = round(0.1 * number)

    # Share from target community top betweenness nodes
    print("Target community: ", G.nodes[start_node]['modularity_class'])
    target_community_nodes = find_by_modularity_class(G.nodes[start_node]['modularity_class'], G)
    target_community_nodes.remove(start_node)
    community_hbt_share = min(community_hbt_share, len(target_community_nodes))
    tops = find_top_high_measure_list(target_community_nodes, G, community_hbt_share ,'betweenness_centrality')
    top_high_fashaness.update(tops)
    
    # Share from community bridge nodes
    all_possible_bridges = set()
    bridge_nodes = []
    for node in target_community_nodes:
        incoming_neighbors = list(G.predecessors(node))
        all_possible_bridges.update(incoming_neighbors)
    for node in all_possible_bridges:
        if node not in target_community_nodes:
            bridge_nodes.append(node)
    bridge_nodes = find_top_bridges(G, bridge_nodes)
    community_brg_share = min(community_brg_share, len(bridge_nodes))
    top_high_fashaness.update(
        node for node in bridge_nodes
        if node not in top_high_fashaness and len(top_high_fashaness) <= (community_brg_share + community_hbt_share)
    )
    
    # Share from total graph top betweenness nodes
    tops = find_top_high_measure(G, len(G.nodes()) ,'betweenness_centrality')
    top_high_fashaness.update(
        node for node in tops
        if node not in top_high_fashaness and len(top_high_fashaness) <= number
    )
    
    return list(top_high_fashaness)

In [135]:
def find_top_high_fashaness(G, start_node, number):
    top_high_fashaness = set()

    # Calculate shares
    community_hbt_share = round(0.5 * number)
    community_brg_share = round(0.4 * number)
    totalgraph_hbt_share = round(0.1 * number)

    # Share from target community top betweenness nodes
    # print("Target community: ", G.nodes[start_node]['modularity_class'])
    target_community_nodes = find_by_modularity_class(G.nodes[start_node]['modularity_class'], G)
    target_community_nodes.remove(start_node)
    community_hbt_share = min(community_hbt_share, len(target_community_nodes))
    tops = find_top_high_measure_list(target_community_nodes, G, community_hbt_share, 'betweenness_centrality')
    top_high_fashaness.update(tops)

    
    # Share from community bridge nodes
    all_possible_bridges = set()
    bridge_nodes = []
    for node in target_community_nodes:
        incoming_neighbors = list(G.predecessors(node))
        all_possible_bridges.update(incoming_neighbors)
    for node in all_possible_bridges:
        if node not in target_community_nodes:
            bridge_nodes.append(node)
    bridge_nodes = find_top_bridges(G, bridge_nodes)
    community_brg_share = min(community_brg_share, len(bridge_nodes))
    for node in bridge_nodes:
        if len(top_high_fashaness) < (community_hbt_share + community_brg_share):
            if node not in top_high_fashaness:
                top_high_fashaness.add(node)

    
    # Share from total graph top betweenness nodes
    tops = find_top_high_measure(G, len(G.nodes()), 'betweenness_centrality')
    for node in tops:
        if len(top_high_fashaness) < number:
            if node not in top_high_fashaness:
                top_high_fashaness.add(node)

    return list(top_high_fashaness)

In [133]:
a = find_top_high_fashaness(G, 1132309696, 10)

Target community:  189


In [150]:
def compare_centrality_measures(length, G):
    seed_nodes = find_refference_nodes(G,1)
    betweenness_disable_diffs = importance_disable_diffs = closeness_disable_diffs = fasha_disable_diffs = 0

    top_high_betweenness = find_top_high_measure(G, 100,'betweenness_centrality')
    top_high_importance = find_top_high_measure(G, 100, 'importance')
    top_high_closeness = find_top_high_measure(G, 100, 'closeness_centrality')
    top_high_importance = find_top_high_measure(G, 100, 'importance')
    # top_high_fasha = find_top_high_fashaness(G, start_node , 100)
    
    for i, node in enumerate(random.sample(seed_nodes, length)):

        top_high_fasha = find_top_high_fashaness(G, node , 100)
        
        start_node = node
        print(i+1, "-",G.nodes[start_node]['Label'])
        
        # normal diffusion
        normal_spread_count = simulate_spread(G, start_node, visualization=False, iterations=30)
        
        # diffusion after disabling top high betweenness node
        disabled_high_betweenness_spread_count = simulate_spread(G, start_node, disabled_nodes=top_high_betweenness, visualization=False, iterations=30)
    
        # diffusion after disabling top high importance node        
        disabled_high_importance_spread_count = simulate_spread(G, start_node, disabled_nodes=top_high_importance, visualization=False, iterations=30)
        
        # diffusion after disabling top high closeness node
        disabled_high_closeness_spread_count = simulate_spread(G, start_node, disabled_nodes=top_high_closeness, visualization=False, iterations=30)

        # diffusion after disabling top high fashaness node
        disabled_high_fasha_spread_count = simulate_spread(G, start_node, disabled_nodes=top_high_fasha, visualization=False, iterations=30)
    
        # comparision
        mean_reach = sum(normal_spread_count)/len(normal_spread_count)
        mean_reach_disabled = sum(disabled_high_betweenness_spread_count)/len(disabled_high_betweenness_spread_count)
        mean_reach_disabled2 = sum(disabled_high_importance_spread_count)/len(disabled_high_importance_spread_count)
        mean_reach_disabled3 = sum(disabled_high_closeness_spread_count)/len(disabled_high_closeness_spread_count)
        mean_reach_disabled4 = sum(disabled_high_fasha_spread_count)/len(disabled_high_fasha_spread_count)
        
        print("Diffusion Result:", normal_spread_count, "mean reach: ",round(mean_reach))
        print("After disabling top Betweenness nodes:", disabled_high_betweenness_spread_count,  "mean reach: ",round(mean_reach_disabled,2))
        print("After disabling top Importance nodes:", disabled_high_importance_spread_count,  "mean reach: ",round(mean_reach_disabled2,2))
        print("After disabling top Closeness nodes:", disabled_high_closeness_spread_count,  "mean reach: ",round(mean_reach_disabled3,2))
        print("After disabling top Fasha nodes:", disabled_high_fasha_spread_count,  "mean reach: ",round(mean_reach_disabled4,2))
    
        betweenness_disable_diffs += (mean_reach - mean_reach_disabled)
        importance_disable_diffs += (mean_reach - mean_reach_disabled2)
        closeness_disable_diffs += (mean_reach - mean_reach_disabled3)
        fasha_disable_diffs += (mean_reach - mean_reach_disabled4)
    
        print("Betweenness disable effect: ", betweenness_disable_diffs)
        print("Importance disable effect: ", importance_disable_diffs)
        print("Closeness disable effect: ", closeness_disable_diffs)
        print("Fasha disable effect: ", fasha_disable_diffs)

In [152]:
compare_centrality_measures(10, G)

1 - rahavardeksh
Diffusion Result: [2, 5, 2, 7, 2, 2, 2, 2, 7, 2, 2, 5, 3, 2, 2, 1, 3, 5, 7, 3, 2, 2, 2, 1, 2, 3, 2, 3, 5, 1] mean reach:  3
After disabling top Betweenness nodes: [2, 2, 2, 2, 1, 3, 6, 3, 1, 1, 2, 5, 2, 5, 1, 3, 1, 2, 2, 2, 6, 1, 3, 2, 1, 3, 3, 1, 3, 4] mean reach:  2.5
After disabling top Importance nodes: [2, 2, 2, 2, 2, 6, 1, 6, 2, 2, 6, 5, 1, 3, 2, 5, 7, 2, 2, 4, 2, 6, 2, 1, 7, 3, 1, 3, 2, 3] mean reach:  3.13
After disabling top Closeness nodes: [1, 1, 2, 2, 4, 2, 5, 2, 1, 2, 3, 3, 6, 1, 2, 2, 6, 1, 2, 3, 5, 6, 2, 1, 1, 1, 1, 6, 1, 2] mean reach:  2.57
After disabling top Fasha nodes: [5, 2, 6, 1, 2, 5, 1, 1, 2, 6, 2, 2, 1, 1, 2, 3, 1, 2, 4, 3, 2, 2, 4, 3, 3, 1, 2, 2, 1, 5] mean reach:  2.57
Betweenness disable effect:  0.4666666666666668
Importance disable effect:  -0.16666666666666652
Closeness disable effect:  0.3999999999999999
Fasha disable effect:  0.3999999999999999
2 - nasa_farsii2
Diffusion Result: [2, 2, 1, 2, 1, 1, 1, 2, 2, 1, 2, 2, 3, 1, 3, 1, 2, 2, 2,

In [153]:
compare_centrality_measures(100, G)

1 - khoycity_iran
Diffusion Result: [1, 1, 1, 28, 1, 2, 2, 5, 4, 2, 1, 2, 2, 1, 3, 3, 4, 4, 2, 5, 3, 3, 4, 13, 2, 1, 6, 2, 5, 1] mean reach:  4
After disabling top Betweenness nodes: [8, 3, 1, 5, 4, 2, 1, 8, 9, 3, 1, 1, 3, 3, 6, 7, 2, 1, 1, 3, 4, 4, 12, 4, 2, 1, 2, 2, 1, 3] mean reach:  3.57
After disabling top Importance nodes: [20, 1, 1, 3, 1, 1, 16, 3, 5, 4, 1, 1, 1, 2, 8, 3, 1, 3, 2, 3, 4, 4, 2, 2, 3, 1, 1, 5, 1, 2] mean reach:  3.5
After disabling top Closeness nodes: [1, 1, 4, 2, 1, 1, 4, 3, 14, 1, 1, 2, 6, 2, 3, 3, 5, 1, 6, 3, 14, 2, 1, 1, 8, 1, 1, 4, 3, 6] mean reach:  3.5
After disabling top Fasha nodes: [2, 4, 1, 2, 2, 1, 2, 1, 6, 1, 1, 1, 1, 1, 4, 3, 9, 3, 9, 1, 1, 1, 1, 9, 3, 2, 1, 5, 6, 2] mean reach:  2.87
Betweenness disable effect:  0.23333333333333295
Importance disable effect:  0.2999999999999998
Closeness disable effect:  0.2999999999999998
Fasha disable effect:  0.9333333333333331
2 - tv1irib_sima
Diffusion Result: [10, 1, 2, 2, 7, 2, 4, 1, 2, 1, 2, 17, 8, 1, 3, 2, 

In [18]:
len(list(G.nodes()))

8626

In [19]:
compare_centrality_measures(1000, G)

1 - edalatxah
Diffusion Result: [2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 1, 2, 1, 1, 2, 3, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2] mean reach:  2
After disabling top Betweenness nodes: [3, 2, 3, 1, 2, 2, 2, 2, 3, 3, 1, 2, 2, 2, 3, 2, 1, 1, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2] mean reach:  2.13
After disabling top Importance nodes: [3, 2, 3, 1, 3, 2, 3, 3, 3, 2, 3, 2, 2, 2, 2, 3, 1, 2, 1, 1, 3, 3, 3, 3, 2, 1, 2, 3, 1, 1] mean reach:  2.2
After disabling top Closeness nodes: [2, 2, 3, 1, 1, 3, 1, 2, 2, 3, 2, 2, 1, 2, 2, 2, 1, 1, 3, 2, 2, 2, 1, 2, 1, 3, 1, 3, 2, 2] mean reach:  1.9
2 - radioasrjadid
Diffusion Result: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] mean reach:  2
After disabling top Betweenness nodes: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] mean reach:  2.0
After disabling top Importance nodes: [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2] mean 

KeyboardInterrupt: 

In [25]:
from collections import defaultdict

def modularity_classification(noeds_list, target_node, community_limit = 100):
    modularity_classes = dict()
    key_list = []
    for node in noeds_list:
        if node in G:
            modularity_class = G.nodes[node].get('modularity_class')
            edge_weight = G.get_edge_data(node, target_node).get('weight')
            modularity_classes[modularity_class] = modularity_classes.get(modularity_class, 0) + edge_weight 
    for mod_class, weight_sum in modularity_classes.items():
        if(weight_sum < community_limit):
            key_list.append(mod_class)
        else:
            print(f"* Modularity Class {mod_class}: {nodes}")
    return {k: v for k, v in modularity_classes.items() if k not in key_list}

In [136]:
find_top_high_fashaness(G, 1320501930, 10)

[1002765601,
 1006212930,
 1006939659,
 1007007698,
 1013336404,
 1005280116,
 1027676885,
 1001976122,
 1006227643,
 1073422940]